In [2]:
!pip install nltk
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [3]:
# Import necessary libraries
import pandas as pd
import random
from nltk.corpus import wordnet
from google.colab import files

In [4]:
# Load the CSV file and convert it into a dictionary of CEFR levels
def load_cefr_words_from_csv():
    uploaded = files.upload()  # Allows you to upload a file from your local device
    filename = next(iter(uploaded))  # Get the uploaded file name
    df = pd.read_csv(filename)  # Load the CSV into a DataFrame
    cefr_words = {}
    for level in df['CEFR'].unique():
        cefr_words[level] = df[df['CEFR'] == level]['headword'].tolist()
    return cefr_words

In [5]:
# Function to generate MCQ question using WordNet, retrying if no meaning is found
def aiquizgen_generate_question(word, retry_attempts=3):
    attempts = 0

    while attempts < retry_attempts:
        # Get the synsets (word senses) for the given word from WordNet
        synsets = wordnet.synsets(word)

        # If no synsets are found, retry with a new word
        if not synsets:
            print(f"Meaning for '{word}' not found. Retrying with a new word...")
            attempts += 1
            word = random.choice(list(wordnet.words()))  # Pick a new word and retry
            continue

        # Get the definition of the primary sense (first synset)
        correct_answer = synsets[0].definition()

        # Generate related distractors (wrong answers)
        distractors = []

        # Use hypernyms (broader categories) as distractors
        if synsets[0].hypernyms():
            distractors.append(synsets[0].hypernyms()[0].definition())

        # Use hyponyms (more specific examples) as distractors
        if synsets[0].hyponyms():
            distractors.append(synsets[0].hyponyms()[0].definition())

        # Use the definitions of other senses of the same word (if they exist)
        for syn in synsets[1:]:
            if len(distractors) < 3:  # Limit to 3 distractors
                distractors.append(syn.definition())

        # If there aren't enough distractors, fill the rest with random words
        while len(distractors) < 3:
            random_word = random.choice(list(wordnet.words()))
            random_synset = wordnet.synsets(random_word)
            if random_synset and random_synset[0].definition() != correct_answer:
                distractors.append(random_synset[0].definition())

        # Shuffle the correct answer and distractors to form options
        options = [correct_answer] + distractors
        random.shuffle(options)

        return {
            'question': f"What does the word '{word}' mean?",
            'options': options,
            'correct_answer': correct_answer
        }

    # If after retries no valid word is found, return a fallback response
    print(f"Unable to find a valid meaning for any word after {retry_attempts} attempts.")
    return {
        'question': f"Meaning for '{word}' could not be found.",
        'options': [
            "Meaning not found in WordNet",
            "Option 2",
            "Option 3",
            "Option 4"
        ],
        'correct_answer': "Meaning not found in WordNet"
    }


In [6]:
# Function to generate multiple MCQs for a given CEFR level
def generate_mcqs_with_aiquizgen(cefr_words, num_questions_per_level=5):
    questions = []
    for level, words in cefr_words.items():
        for _ in range(num_questions_per_level):
            word = random.choice(list(words))  # Convert the word list to a list before selecting
            # Call AIQuizGen to generate the question and options
            question_data = aiquizgen_generate_question(word)

            question = {
                'level': level,
                'word': word,
                'question_text': question_data['question'],
                'options': question_data['options'],
                'correct_answer': question_data['correct_answer']
            }
            questions.append(question)
    random.shuffle(questions)
    return questions



In [7]:
# Function to ask MCQs and get user responses with input validation
def ask_mcqs(questions):
    user_responses = []
    print("Please answer the following questions:")
    for i, question in enumerate(questions):
        print(f"\nQuestion {i+1}: {question['question_text']}")
        for idx, option in enumerate(question['options']):
            print(f"{idx+1}. {option}")

        # Validate user input
        while True:
            user_choice = input("Your answer (1/2/3/4): ")
            if user_choice in ['1', '2', '3', '4']:
                break  # Valid input, exit the loop
            else:
                print("Invalid input. Please choose a number between 1 and 4.")

        user_responses.append({
            'question': question,
            'user_answer': question['options'][int(user_choice)-1]
        })
    return user_responses

In [8]:
# Improved feedback function after each level
def evaluate_responses(user_responses):
    score_per_level = {}
    detailed_feedback = []  # Store detailed feedback for each question

    for response in user_responses:
        level = response['question']['level']
        correct_answer = response['question']['correct_answer']
        user_answer = response['user_answer']

        if level not in score_per_level:
            score_per_level[level] = {'correct': 0, 'total': 0}

        score_per_level[level]['total'] += 1
        if user_answer == correct_answer:
            score_per_level[level]['correct'] += 1
            detailed_feedback.append(f"Q: {response['question']['question_text']} - Correct!")
        else:
            detailed_feedback.append(f"Q: {response['question']['question_text']} - Incorrect. Correct answer: {correct_answer}")

    # Print detailed feedback
    print("\nDetailed Feedback:")
    for feedback in detailed_feedback:
        print(feedback)

    return score_per_level


In [9]:
# Function to determine the user's CEFR level based on their performance
def determine_user_level(score_per_level):
    next_level_threshold = 80  # User needs 80% or more to move to the next level
    stay_level_threshold = 60  # User needs at least 60% to stay at the current level
    cefr_levels = ['A1', 'A2', 'B1', 'B2', 'C1', 'C2']

    for level in reversed(cefr_levels):
        if level in score_per_level:
            correct = score_per_level[level]['correct']
            total = score_per_level[level]['total']
            accuracy = correct / total * 100

            if accuracy >= next_level_threshold:
                print(f"Level {level}: {correct}/{total} correct ({accuracy:.2f}% accuracy) - Advancing to next level!")
                return level  # Return the next level
            elif accuracy < stay_level_threshold:
                print(f"Level {level}: {correct}/{total} correct ({accuracy:.2f}% accuracy) - Stay at the current level!")
                break  # Stop checking as the user cannot advance
            else:
                print(f"Level {level}: {correct}/{total} correct ({accuracy:.2f}% accuracy) - Staying at this level!")
                return level  # User is staying at this level

    return cefr_levels[0]  # If no level is achieved, return the lowest level (A1)


In [10]:
# Storing user progress after each session
def store_user_progress(user_responses, user_level):
    progress_data = {
        'responses': user_responses,
        'final_level': user_level
    }
    # Saving progress to a text file
    with open('user_progress.txt', 'w') as file:
        file.write(str(progress_data))
    print("User progress has been saved.")


In [ ]:
# Main function with adaptive difficulty
def main():
    # Load CEFR words from the uploaded file
    cefr_words = load_cefr_words_from_csv()

    # Ask the user for their starting level or default to A1
    cefr_levels = ['A1', 'A2', 'B1', 'B2', 'C1', 'C2']
    print("Available levels: A1, A2, B1, B2, C1, C2")
    starting_level = input("Please select a starting level (e.g., A1): ").upper()
    if starting_level not in cefr_levels:
        print("Invalid input. Starting at A1.")
        current_level_index = 0
    else:
        current_level_index = cefr_levels.index(starting_level)

    while current_level_index < len(cefr_levels):
        current_level = cefr_levels[current_level_index]
        print(f"\nStarting level: {current_level}")

        # Generate MCQs for the current level
        questions = generate_mcqs_with_aiquizgen({current_level: cefr_words[current_level]}, num_questions_per_level=5)

        # Ask MCQs and get user responses
        user_responses = ask_mcqs(questions)

 # Evaluate user responses and get score per level
        score_per_level = evaluate_responses(user_responses)

        # Determine the user's level based on performance
        user_level = determine_user_level(score_per_level)

        # Store user progress
        store_user_progress(user_responses, user_level)

        # Check if the user should advance to the next level
        if user_level == current_level:
            print(f"You're staying at the {current_level} level.")
            break  # User stays at this level or lower, exit the loop
        else:
            print(f"Congratulations! You're advancing to the next level: {cefr_levels[current_level_index + 1]}")
            current_level_index += 1  # Move to the next level

# Run the main function
if __name__ == "__main__":
    main()









Saving ENGLISH_CERF_WORDS.csv to ENGLISH_CERF_WORDS.csv
Available levels: A1, A2, B1, B2, C1, C2
